# Creamos una Funcion para desarmar Excel y separar sus hojas para trabajar por separado

In [20]:
import pandas as pd
import os

rutaEntrada= r"C:\Users\Pedro\Desktop\DAProyect\Telecomunicaciones\Internet.xlsx"
rutaSalida= r"C:\Users\Pedro\Desktop\DAProyect\Telecomunicaciones\HojasSeparadas"

In [21]:

def split_excel_sheets(rutaEntrada, rutaSalida):
    """
    Parámetros:
    - rutaEntrada (str): Ruta completa del archivo Excel.
    - rutaSalida(str): Directorio donde se guardarán las hojas separadas.

    Retorno:
    - None: Crea archivos separados en el directorio especificado.
    """
    try:
        # Lee todas las hojas del archivo Excel
        excel_data = pd.ExcelFile(rutaEntrada)
        sheet_names = excel_data.sheet_names
        
        # Crea el directorio de salida si no existe
        if not os.path.exists(rutaSalida):
            os.makedirs(rutaSalida)
        
        # Guarda cada hoja como un archivo CSV
        for sheet in sheet_names:
            df = excel_data.parse(sheet)
            output_file = os.path.join(rutaSalida, f"{sheet}.csv")
            df.to_csv(output_file, index=False, encoding='utf-8')
            print(f"Guardado: {output_file}")
        
        print("Todas las hojas se han separado exitosamente.")
    
    except Exception as e:
        print(f"Error al procesar el archivo: {e}")

# Llamar a la función
split_excel_sheets(rutaEntrada, rutaSalida)


Guardado: C:\Users\Pedro\Desktop\DAProyect\Telecomunicaciones\HojasSeparadas\Acc_vel_loc_sinrangos.csv
Guardado: C:\Users\Pedro\Desktop\DAProyect\Telecomunicaciones\HojasSeparadas\Accesos_tecnologia_localidad.csv
Guardado: C:\Users\Pedro\Desktop\DAProyect\Telecomunicaciones\HojasSeparadas\Totales Accesos Por Tecnología.csv
Guardado: C:\Users\Pedro\Desktop\DAProyect\Telecomunicaciones\HojasSeparadas\Accesos Por Tecnología.csv
Guardado: C:\Users\Pedro\Desktop\DAProyect\Telecomunicaciones\HojasSeparadas\Poblacion por provincia y año.csv
Guardado: C:\Users\Pedro\Desktop\DAProyect\Telecomunicaciones\HojasSeparadas\Población por rango etaria.csv
Guardado: C:\Users\Pedro\Desktop\DAProyect\Telecomunicaciones\HojasSeparadas\Velocidad_sin_Rangos.csv
Guardado: C:\Users\Pedro\Desktop\DAProyect\Telecomunicaciones\HojasSeparadas\Velocidad % por prov.csv
Guardado: C:\Users\Pedro\Desktop\DAProyect\Telecomunicaciones\HojasSeparadas\Totales VMD.csv
Guardado: C:\Users\Pedro\Desktop\DAProyect\Telecomunica

# Unimos varios CSV en una única Tabla

In [ ]:
# Ruta de la carpeta con los archivos CSV
ruta_carpeta = r"C:\Users\Pedro\Desktop\DAProyect\Telecomunicaciones\Provincias\query"

# Crear una lista para almacenar los DataFrames
dataframes = []

# Recorrer todos los archivos en la carpeta
for archivo in os.listdir(ruta_carpeta):
    if archivo.endswith('.csv'):  # Verificar que el archivo sea CSV
        # Crear la ruta completa del archivo
        ruta_archivo = os.path.join(ruta_carpeta, archivo)
        
        # Leer el archivo CSV en un DataFrame
        df = pd.read_csv(ruta_archivo)
        
        # Extraer el nombre de la provincia (sin "(2)")
        provincia = archivo.split('(')[0].strip()
        
        # Agregar la columna "Provincia" con el nombre de la provincia
        df['Provincia'] = provincia
        
        # Agregar el DataFrame a la lista
        dataframes.append(df)

# Combinar todos los DataFrames en uno solo
tabla_unica = pd.concat(dataframes, ignore_index=True)

# Guardar el resultado en un archivo CSV
ruta_salida = r"C:\Users\Pedro\Desktop\DAProyect\Telecomunicaciones\Provincias\tabla_unificada.csv"
tabla_unica.to_csv(ruta_salida, index=False, encoding='utf-8')

print(f"Archivo unificado guardado en: {ruta_salida}")



Archivo unificado guardado en: C:\Users\Pedro\Desktop\DAProyect\Telecomunicaciones\Provincias\tabla_unificada.csv


# Corregimos errores de codificación

In [2]:
def corregir_errores_archivo(ruta_archivo, columna, diccionario_correcciones):
    """
    Corrige errores de escritura en una columna específica de un archivo CSV o Excel.
    
    Args:
        ruta_archivo (str): Ruta del archivo a corregir.
        columna (str): Nombre de la columna que contiene los errores.
        diccionario_correcciones (dict): Diccionario con caracteres incorrectos como claves y correcciones como valores.
    
    Returns:
        pd.DataFrame: El DataFrame con la columna corregida.
    """
    # Detectar formato del archivo
    if ruta_archivo.endswith('.csv'):
        dataframe = pd.read_csv(ruta_archivo)
    elif ruta_archivo.endswith(('.xls', '.xlsx')):
        dataframe = pd.read_excel(ruta_archivo)
    else:
        raise ValueError("Formato de archivo no soportado. Solo se aceptan CSV o Excel.")

    # Crear función de corrección basada en el diccionario
    def corregir_texto(texto):
        if pd.isna(texto):  # Manejar valores NaN
            return texto
        for error, correccion in diccionario_correcciones.items():
            texto = texto.replace(error, correccion)
        return texto

    # Aplicar correcciones a la columna especificada
    if columna in dataframe.columns:
        dataframe[columna] = dataframe[columna].apply(corregir_texto)
    else:
        raise ValueError(f"La columna '{columna}' no se encuentra en el archivo.")

    return dataframe

# Diccionario de correcciones
diccionario_correcciones = {
    "Ã¡": "a",
    "Ã©": "e",
    "Ã­­": "i",
    "Ã³": "o",
    "Ãš": "u",
    "Ã±": "ni",
    "Ã¼": "u"
}




ruta_archivo = "C:/Users/Pedro/Desktop/correcto/provincias.csv"  
columna_con_errores = "Provincia"  



# Corregir y guardar el resultado
try:
    df_corregido = corregir_errores_archivo(ruta_archivo, columna_con_errores, diccionario_correcciones)
    print("DataFrame corregido:")
    print(df_corregido.head())
    
    # Determinar el nombre del archivo corregido
    ruta_directorio, nombre_archivo = os.path.split(ruta_archivo)
    nombre_base, extension = os.path.splitext(nombre_archivo)
    nuevo_nombre = f"{nombre_base}_corregido{extension}"
    nueva_ruta = os.path.join(ruta_directorio, nuevo_nombre)
    
    # Guardar el archivo corregido
    if ruta_archivo.endswith('.csv'):
        df_corregido.to_csv(nueva_ruta, index=False)
    elif ruta_archivo.endswith(('.xls', '.xlsx')):
        df_corregido.to_excel(nueva_ruta, index=False)
    
    print(f"Archivo corregido guardado en: {nueva_ruta}")
except ValueError as e:
    print(f"Error: {e}")




DataFrame corregido:
   id_prov                        Provincia   Longitud    Latitud
0        5  Ciudad Autónoma de Buenos Aires -72.233024 -30.063132
1        5  Ciudad Autónoma de Buenos Aires -59.487341 -33.221864
2        5  Ciudad Autónoma de Buenos Aires -65.532131 -27.181367
3        5  Ciudad Autónoma de Buenos Aires -58.394730 -54.176929
4        5  Ciudad Autónoma de Buenos Aires -54.616269 -51.551783
Archivo corregido guardado en: C:/Users/Pedro/Desktop/correcto\provincias_corregido.csv


In [3]:

df= pd.read_csv(r"C:\Users\Pedro\Desktop\correcto\PoblacionRangoEdad.csv")


# Verificamos que los archivos CSV unificados en uno sólo, haya quedado correcto

In [4]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Edad       504 non-null    object
 1   2010       504 non-null    int64 
 2   2011       504 non-null    int64 
 3   2012       504 non-null    int64 
 4   2013       504 non-null    int64 
 5   2014       504 non-null    int64 
 6   2015       504 non-null    int64 
 7   2016       504 non-null    int64 
 8   2017       504 non-null    int64 
 9   2018       504 non-null    int64 
 10  2019       504 non-null    int64 
 11  2020       504 non-null    int64 
 12  2021       504 non-null    int64 
 13  2022       504 non-null    int64 
 14  2023       504 non-null    int64 
 15  2024       504 non-null    int64 
 16  Provincia  504 non-null    object
dtypes: int64(15), object(2)
memory usage: 67.1+ KB


In [9]:
df.head(24)

,Edad,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,Provincia
0,0-4,1371305,1400916,1426126,1445568,1457653,1460832,1451894,1442629,1433249,1424041,1415241,1406898,1399003,1391680,1385047,BUENOS AIRES
1,5-9,1305795,1315159,1324964,1337681,1354522,1376276,1405851,1431033,1450465,1462539,1465713,1456774,1447502,1438121,1428898,BUENOS AIRES
2,10-14,1267369,1271578,1280116,1290242,1300417,1310129,1319458,1329222,1341896,1358686,1380387,1409895,1435021,1454394,1466415,BUENOS AIRES
3,15-19,1295484,1290062,1283109,1276462,1272267,1272046,1276242,1284755,1294844,1304980,1314648,1323938,1333655,1346280,1363012,BUENOS AIRES
4,20-24,1280481,1295159,1305584,1311441,1312573,1308866,1303438,1296477,1289818,1285594,1285326,1289452,1297883,1307876,1317909,BUENOS AIRES
5,25-29,1209580,1219643,1235125,1254326,1274528,1293661,1308258,1318620,1324425,1325526,1321792,1316348,1309360,1302664,1298388,BUENOS AIRES
6,30-34,1205936,1214113,1214704,1211454,1209688,1213193,1223265,1238718,1257869,1278015,1297093,1311644,1321978,1327770,1328868,BUENOS AIRES
7,35-39,1069833,1100865,1132734,1162745,1187436,1204570,1212809,1213512,1210386,1208725,1212304,1222404,1237848,1256964,1277072,BUENOS AIRES
8,40-44,936683,956554,980216,1006951,1035671,1065341,1096284,1128053,1157979,1182628,1199776,1208092,1208930,1205966,1204450,BUENOS AIRES
9,45-49,877839,885490,892761,901063,912293,927722,947551,971129,997737,1026308,1055824,1086606,1118204,1147978,1172528,BUENOS AIRES
